In [0]:
 show catalogs;
 --DESCRIBE CATALOG EXTENDED fleet_raw;



In [0]:
USE CATALOG fleet;
USE SCHEMA bronze;

CREATE TABLE IF NOT EXISTS telemetry_raw (
  vehicle_id   STRING,
  ts_utc       TIMESTAMP,
  speed_kmh    DOUBLE,
  engine_temp  DOUBLE,
  raw_payload  STRING   -- keep the original JSON if you want
)
USING DELTA
LOCATION 'abfss://databricks@iyaddatabricksstg.dfs.core.windows.net/lake/fleet/bronze/telemetry_raw';


USE SCHEMA silver;

CREATE TABLE IF NOT EXISTS telemetry_clean
USING DELTA
LOCATION 'abfss://databricks@iyaddatabricksstg.dfs.core.windows.net/lake/fleet/silver/telemetry_clean'
AS
SELECT
  vehicle_id,
  ts_utc,
  speed_kmh,
  engine_temp
FROM fleet.bronze.telemetry_raw
WHERE ts_utc IS NOT NULL
  AND speed_kmh BETWEEN 0 AND 250;



USE SCHEMA gold;

CREATE TABLE IF NOT EXISTS daily_speed_stats
USING DELTA
LOCATION 'abfss://databricks@iyaddatabricksstg.dfs.core.windows.net/lake/fleet/gold/daily_speed_stats'
AS
SELECT
  vehicle_id,
  DATE(ts_utc) AS day,
  AVG(speed_kmh) AS avg_speed,
  MAX(speed_kmh) AS max_speed
FROM fleet.silver.telemetry_clean
GROUP BY vehicle_id, DATE(ts_utc);
